## General prerprocess data for baseline models.

## Preprocess for both SARIMA and HWES, we need fixed time interval:

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller

In [2]:
df = pd.read_parquet('../data/processed/cleaned.parquet')
price_cols_to_drop = [col for col in df.columns if col.startswith("price_") and col != "price_USD"]
df.drop(columns=price_cols_to_drop, inplace=True) #Delete all price rows except usd
df.head()
df.tail()

,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,mempool_fee_histogram_bin_70_75,mempool_fee_histogram_bin_75_80,mempool_fee_histogram_bin_80_85,mempool_fee_histogram_bin_85_90,mempool_fee_histogram_bin_90_95,mempool_fee_histogram_bin_95_100,mempool_fee_histogram_bin_100_150,mempool_fee_histogram_bin_150_200,mempool_fee_histogram_bin_200_250,mempool_fee_histogram_bin_250_300
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-04-15 09:34:42,1731329.0,997991.75,3810.0,4524017.0,2.149812,3.0,3.0,3.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:39:41,1593473.0,997958.75,3874.0,3061643.0,2.000000,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:44:41,1732510.0,997988.50,4511.0,4356829.0,1.554327,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:49:42,1768397.0,997952.00,4938.0,2320204.0,1.433566,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:54:42,1734405.0,997926.75,4746.0,3605544.0,2.000000,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Check if all the time interval are fixed.

In [3]:
time_diffs = df.index.to_series().diff()
print("Unique time intervals:")
print(time_diffs.value_counts())

most_common_interval = time_diffs.mode()[0]
print(f"\nMost common interval: {most_common_interval}")


Unique time intervals:
timestamp
0 days 00:05:00    4291
0 days 00:05:01    3555
0 days 00:04:59    3555
0 days 00:05:04     128
0 days 00:04:56     127
0 days 00:05:03     110
0 days 00:04:57     110
0 days 00:04:55       3
0 days 00:05:05       2
0 days 00:03:32       1
0 days 00:07:00       1
0 days 00:01:15       1
0 days 00:08:46       1
0 days 00:01:21       1
0 days 00:06:23       1
0 days 00:08:38       1
0 days 00:03:03       1
0 days 00:06:29       1
0 days 00:05:57       1
0 days 00:04:58       1
0 days 00:04:07       1
0 days 00:02:57       1
0 days 00:07:03       1
0 days 00:04:44       1
0 days 00:05:13       1
0 days 00:02:43       1
0 days 00:07:17       1
0 days 00:03:36       1
Name: count, dtype: int64

Most common interval: 0 days 00:05:00


In [4]:
# Resample to 30-minute regular intervals, averaging or interpolating values
# No actual missing data, just values not aligned exactly on the desired resampling grid, so we can interpolate the data.
df_resampled = df.resample("30min").mean()
# Linear interpolation is safe here due to small timing variations
df_resampled = df_resampled.interpolate(method='linear')
df_resampled.head()
df_resampled.tail()

,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,mempool_fee_histogram_bin_70_75,mempool_fee_histogram_bin_75_80,mempool_fee_histogram_bin_80_85,mempool_fee_histogram_bin_85_90,mempool_fee_histogram_bin_90_95,mempool_fee_histogram_bin_95_100,mempool_fee_histogram_bin_100_150,mempool_fee_histogram_bin_150_200,mempool_fee_histogram_bin_200_250,mempool_fee_histogram_bin_250_300
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-04-15 07:30:00,1.436588e+06,815788.750000,2542.166667,4.146425e+06,1.715012,2.333333,2.166667,1.333333,1.166667,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2025-04-15 08:00:00,8.725617e+05,481176.041667,1620.833333,3.306669e+06,0.520950,1.000000,1.000000,1.000000,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9837.000000,0.0,0.0,0.0
2025-04-15 08:30:00,1.563087e+06,886380.541667,3235.166667,5.851361e+06,1.322925,2.000000,1.333333,1.000000,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11472.333333,0.0,0.0,0.0
2025-04-15 09:00:00,1.771240e+06,997961.583333,3899.000000,1.626690e+07,4.163795,5.000000,4.000000,3.333333,2.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,108493.833333,0.0,0.0,0.0
2025-04-15 09:30:00,1.712023e+06,997963.550000,4375.800000,3.573647e+06,1.827541,2.200000,2.200000,2.200000,2.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


## Preprocess for XGboost

In [5]:
#We will need lagging feature for XGboost. Since each hour has 2 data points (60 ÷ 30), 48 hours = 96 lag steps.
def create_lag_features_fast(df, target_col, lags):
    lagged_dfs = [
        df[[target_col]].shift(lag).rename(columns={target_col: f'{target_col}_lag_{lag}'})
        for lag in lags
    ]
    return pd.concat([df] + lagged_dfs, axis=1)
lags = range(1, 97)  # 48 hours of 5-minute intervals
df_xgboost = create_lag_features_fast(df_resampled, 'recommended_fee_fastestFee', lags)
df_xgboost.head()

,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,recommended_fee_fastestFee_lag_87,recommended_fee_fastestFee_lag_88,recommended_fee_fastestFee_lag_89,recommended_fee_fastestFee_lag_90,recommended_fee_fastestFee_lag_91,recommended_fee_fastestFee_lag_92,recommended_fee_fastestFee_lag_93,recommended_fee_fastestFee_lag_94,recommended_fee_fastestFee_lag_95,recommended_fee_fastestFee_lag_96
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-03-05 02:00:00,1.884101e+06,997913.000000,2227.750000,3.075377e+06,2.180505,2.750000,2.500000,2.500000,2.500000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 02:30:00,2.031075e+06,997968.000000,1109.333333,2.011305e+06,1.445995,2.166667,2.166667,2.166667,2.166667,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 03:00:00,1.694918e+06,997964.500000,3056.000000,4.435382e+06,3.062388,3.833333,3.166667,3.000000,3.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 03:30:00,1.765702e+06,997931.916667,1778.166667,4.142780e+06,2.692898,3.500000,3.166667,2.833333,2.833333,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 04:00:00,1.903513e+06,997951.541667,1716.000000,2.664654e+06,1.744798,2.500000,2.333333,2.333333,2.333333,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_xgboost.to_parquet("../data/processed/preprocessed_xgboost_30min.parquet")